In [113]:
import gymnasium as gym
import numpy as np
from collections import defaultdict
from typing import Any
import math

env = gym.make("Blackjack-v1")

In [114]:
# Definition of a random starting policy
example_policy = defaultdict(lambda: {env.action_space.sample(): 1.0})

# Retrieval function for policies
def get_action_from_policy(policy: dict[Any, dict[Any, float]], state: Any) -> float:
    return np.random.choice(a=list(policy[state].keys()),p=list(policy[state].values()))

In [115]:
# Example execution loop
done = False
total_reward = 0

observation, _ = env.reset()

while not done:
    action = get_action_from_policy(example_policy, observation)

    observation, reward, terminated, truncated, info = env.step(action)
    total_reward += reward

    done = terminated or truncated

total_reward

-1.0

In [116]:
# Some utils
def generate_episode(env: gym.Env, policy):
    done = False
    state, _ = env.reset()
    tuples = []
    while not done:
        action = get_action_from_policy(policy, state)
        state_, reward, terminated, truncated, info = env.step(action)

        tuples.append((state, int(action), reward))

        state = state_
        done = terminated or truncated
    
    return tuples

def generate_episode_exploring_starts(env: gym.Env, policy):
    env.reset()
    env.unwrapped.s = env.observation_space.sample()

    initial_state = env.unwrapped.s
    initial_action = env.action_space.sample()

    tuples = []
    state = initial_state
    action = initial_action

    while True:
        next_state, reward, terminated, truncated, _ = env.step(action)
        tuples.append((state,action,reward))

        if terminated or truncated:
            break

        state = next_state
        action = get_action_from_policy(policy, state)

    return tuples

In [117]:
def first_visit_MC_prediction(env: gym.Env, policy, episodes:int=10_000, gamma: float = 0.9):
    values = defaultdict(float)
    counts = defaultdict(int)

    for _ in range(episodes):
        episode = generate_episode(env, policy)
        g = 0

        # Pre-calculate first indices
        first_visit_idx = {}
        for idx, (state,_,_) in enumerate(episode):
            if state not in first_visit_idx:
                first_visit_idx[state] = idx

        for i in range(len(episode) -1, -1, -1):
            state, action, reward = episode[i]
            g = gamma * g + reward

            # Checks if this is truly first visit
            if i == first_visit_idx.get(state):
                counts[state] += 1

                # Avg mean update, like in multi armed bandits
                values[state] += (g - values[state]) / counts[state] 

    return values

first_visit_MC_prediction(env, example_policy)

defaultdict(float,
            {(19, 10, 0): -0.8270781893004111,
             (19, 9, 1): 0.06666666666666668,
             (8, 5, 0): -0.3333333333333333,
             (21, 4, 1): 0.9736842105263157,
             (13, 9, 0): -0.46875,
             (15, 9, 1): -0.33,
             (16, 1, 0): -0.9250000000000002,
             (16, 3, 0): -0.8534999999999998,
             (8, 3, 0): -0.5778947368421055,
             (13, 10, 0): -0.5323076923076917,
             (12, 2, 0): -0.3763440860215053,
             (8, 10, 0): -0.5604395604395604,
             (9, 10, 0): -0.5272727272727271,
             (20, 10, 0): -0.8945945945945946,
             (17, 10, 0): -0.8190999999999996,
             (14, 10, 0): -0.6677740863787377,
             (12, 9, 0): -0.4970833333333332,
             (13, 5, 0): -0.1739130434782608,
             (18, 9, 0): -0.0632911392405063,
             (20, 9, 1): -0.3,
             (19, 3, 1): 0.2631578947368421,
             (16, 10, 0): -0.5910780669144976,
       

In [118]:
# Exploring Starts and following GPI: Evaluation -> Improvement
def monte_carlo_ES(env: gym.Env, episodes:int=10_000, gamma: float = 0.9):
    policy = defaultdict(lambda: {env.action_space.sample(): 1.0})
    q_values = defaultdict(float)
    counts = defaultdict(int)

    for _ in range(episodes):
        episode = generate_episode_exploring_starts(env, policy)
        g = 0

        # Pre-calculate first indices
        first_visit_idx = {}
        for idx, (state,action,_) in enumerate(episode):
            if (state,action) not in first_visit_idx:
                first_visit_idx[(state,action)] = idx

        for i in range(len(episode) -1, -1, -1):
            state, action, reward = episode[i]
            g = gamma * g + reward

            # Checks if this is truly first visit
            if i == first_visit_idx.get((state,action)):
                counts[(state,action)] += 1

                # Avg mean update, like in multi armed bandits
                q_values[(state,action)] += (g - q_values[(state,action)]) / counts[(state,action)] 

                # Policy improvement step
                max_a = env.action_space.sample()
                max_q_val = q_values[(state,max_a)]
                for a in range(env.action_space.n):
                    if q_values[(state,a)] > max_q_val:
                        max_q_val = q_values[(state,a)]
                        max_a = a

                policy[state] = {max_a: 1.0}

    return policy

monte_carlo_ES_optimized_policy = monte_carlo_ES(env, episodes=100_000, gamma=1.0)

In [119]:
# We benchmark policies
def get_avg_reward(env: gym.Env, policy, iterations=10_000):
    total_reward = 0
    for _ in range(iterations):
        done = False
        observation, _ = env.reset()
        while not done:
            action = get_action_from_policy(policy, observation)
            observation, reward, terminated, truncated, info = env.step(action)
            total_reward += reward
            done = terminated or truncated
    return total_reward / iterations

random_policy_avg_reward = get_avg_reward(env, example_policy) 
print(f"Got {random_policy_avg_reward:.3} from random policy")

MC_ES_policy_avg_reward = get_avg_reward(env, monte_carlo_ES_optimized_policy) 
print(f"Got {MC_ES_policy_avg_reward:.3} from monte carlo exploring starts optimized policy")


Got -0.367 from random policy
Got -0.129 from monte carlo exploring starts optimized policy


In [120]:
# Util for generating a random epsilon-soft policy
def generate_epsilon_soft_policy_on_state(env: gym.Env, epsilon: float):
    policy_on_state = {
        action: epsilon / env.action_space.n for action in range(env.action_space.n)
    }
    policy_on_state[np.random.choice(range(env.action_space.n))] += 1 - epsilon

    return policy_on_state

def assign_epsilon_soft_policy_on_state(env: gym.Env, epsilon: float, action: Any):
    policy_on_state = {
        action: epsilon / env.action_space.n for action in range(env.action_space.n)
    }
    policy_on_state[action] += 1 - epsilon

    return policy_on_state

generate_epsilon_soft_policy_on_state(env, 0.1)

{0: np.float64(0.05), 1: np.float64(0.9500000000000001)}

In [ ]:
# An algorithm for getting the optimal epsilon-soft policy using MC control
def on_policy_first_visit_MC_control(env: gym.Env, episodes:int=10_000, gamma: float = 0.9, epsilon: float = 0.1):
    policy = defaultdict(lambda: generate_epsilon_soft_policy_on_state(env, epsilon))
    q_values = defaultdict(float)
    counts = defaultdict(int)

    for _ in range(episodes):
        episode = generate_episode(env, policy)
        g = 0

        # Pre-calculate first indices
        first_visit_idx = {}
        for idx, (state,action,_) in enumerate(episode):
            if (state,action) not in first_visit_idx:
                first_visit_idx[(state,action)] = idx

        for i in range(len(episode) -1, -1, -1):
            state, action, reward = episode[i]
            g = gamma * g + reward

            # Checks if this is truly first visit
            if i == first_visit_idx.get((state,action)):
                counts[(state,action)] += 1

                # Avg mean update, like in multi armed bandits
                q_values[(state,action)] += (g - q_values[(state,action)]) / counts[(state,action)] 

                # Policy improvement step
                max_a = env.action_space.sample()
                max_q_val = q_values[(state,max_a)]
                for a in range(env.action_space.n):
                    if q_values[(state,a)] > max_q_val:
                        max_q_val = q_values[(state,a)]
                        max_a = a

                policy[state] = assign_epsilon_soft_policy_on_state(env, epsilon, max_a)

    return policy

on_policy_MC_control_optimized_policy = on_policy_first_visit_MC_control(env, episodes=100_000, gamma=.9)

In [122]:
random_policy_avg_reward = get_avg_reward(env, example_policy) 
print(f"Got {random_policy_avg_reward:.3} from random policy")

MC_ES_policy_avg_reward = get_avg_reward(env, monte_carlo_ES_optimized_policy) 
print(f"Got {MC_ES_policy_avg_reward:.3} from monte carlo exploring starts optimized policy")

on_policy_MC_policy_avg_reward = get_avg_reward(env, on_policy_MC_control_optimized_policy)
print(f"Got {on_policy_MC_policy_avg_reward:.3} from monte carlo control on-policy optimized policy")

Got -0.37 from random policy
Got -0.126 from monte carlo exploring starts optimized policy
Got -0.0822 from monte carlo control on-policy optimized policy


In [123]:
def off_policy_mc_prediction(env: gym.Env, b_policy, target_policy, gamma:float=0.9, episodes:int=1000):
    q_values = defaultdict(float)
    c = defaultdict(lambda: 0.0)

    for _ in range(episodes):
        episode = generate_episode(env, b_policy)
        g = 0
        w = 1

        for i in range(len(episode) -1, -1, -1):
            state, action, reward = episode[i]
            g = gamma * g + reward
            c[(state,action)] += w
            q_values[(state,action)] += (w / c[(state,action)]) * (g - q_values[(state,action)])
            w = w * (target_policy[state][action] / b_policy[state][action])
        
    return q_values

example_policy = defaultdict(lambda: generate_epsilon_soft_policy_on_state(env, epsilon=0.1))

off_policy_mc_prediction(env,b_policy=example_policy,target_policy=on_policy_MC_control_optimized_policy, episodes=1000)

defaultdict(float,
            {((20, 3, 0), 1): np.float64(-0.9992537313432835),
             ((15, 2, 0), 1): np.float64(-0.6454545454545454),
             ((10, 2, 0), 1): np.float64(-0.04333333333333328),
             ((16, 8, 0), 0): -0.5,
             ((13, 8, 0), 1): np.float64(-0.5175324675324675),
             ((21, 10, 1), 0): 0.9374999999999999,
             ((10, 10, 0), 1): np.float64(-0.13800000000000012),
             ((20, 1, 0), 1): np.float64(-0.9994186046511628),
             ((21, 1, 1), 1): np.float64(-0.8940681818181818),
             ((20, 10, 0), 1): np.float64(-0.9408163265306122),
             ((12, 10, 0), 1): np.float64(-0.32465967016491754),
             ((17, 8, 0), 0): -1.3877787807814457e-17,
             ((20, 8, 0), 1): np.float64(-0.999476439790576),
             ((19, 6, 0), 0): 1.0,
             ((9, 6, 0), 1): np.float64(0.0),
             ((15, 9, 0), 0): -0.14285714285714288,
             ((17, 10, 0), 0): -0.5714285714285714,
             ((19, 